In [10]:
risk_data = spark.read.format("delta").table("raw_to_coverage_temp").cache()
earned_data = spark.read.format("delta").table("A6_MOTOR_DATE_EARNED_RATE").cache()

joined_data = risk_data.join(
    earned_data,
    ["std_v_date_issue", "std_v_date_expiry"],
    "left"
)

StatementMeta(, 760acdad-592c-47d2-81a6-920f0c22a58b, 12, Finished, Available, Finished)

In [12]:
from pyspark.sql.functions import coalesce
final_data = joined_data.withColumn(
    "EARNED_PREMIUM",
    coalesce(col("EARNED_RATE"), lit(0)) * col("PREMIUM_written")
).withColumn(
    "EARNED_UWE",
    coalesce(col("EARNED_RATE"), lit(0)) * col("UWE_written")
).withColumn(
    "EARNED_CAT",
    coalesce(col("EARNED_RATE"), lit(0)) * col("CAT_written")
).withColumn(
    "is_reverted",
    lit(0)
).select(
    "data_system",
    "v_uid",
    "p_code",
    "p_product_code",
    "std_p_branch",
    "std_p_department",
    "std_p_code_cooperation_type",
    "std_p_customer_name",
    "p_salestaff",
    "coverage",
    "std_v_date_issue",
    "std_v_date_expiry",
    "earn_date",
    "EARNED_PREMIUM",
    "EARNED_UWE",
    "EARNED_CAT",
    "is_reverted"
)


StatementMeta(, 760acdad-592c-47d2-81a6-920f0c22a58b, 14, Finished, Available, Finished)

In [13]:
final_data.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("A6_MOTOR_EARNED_PREMIUM")

StatementMeta(, 760acdad-592c-47d2-81a6-920f0c22a58b, 15, Finished, Available, Finished)